# Módulo 7 - Actividad 5:
# Entrenamiento de un modelo con técnicas de regularización y optimización

## Objetivo
Diseñar, entrenar y evaluar una red neuronal convolucional utilizando al menos dos técnicas de regularización y dos de optimización, ajustadas al contexto de un dataset realista.

**Datasets utilizados:**  
`CIFAR-10`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

> Se diseñó un pipeline modular para entrenar un modelo CNN desde cero sobre el dataset CIFAR-10. El flujo abarca la carga y preprocesamiento de datos, construcción y compilación del modelo, entrenamiento con callbacks para optimización, evaluación y visualización de métricas.

1. **Carga y preprocesamiento de datos:**  
   - Se cargó el dataset CIFAR-10, compuesto por imágenes RGB de 32x32 píxeles en 10 clases.  
   - Se normalizaron las imágenes para escalar los valores de píxeles a un rango [0, 1].  
   - Las etiquetas se convirtieron a formato one-hot para facilitar la clasificación multiclase.  

2. **Construcción del modelo:**  
   - Se definió una arquitectura CNN con dos capas convolucionales seguidas de capas de pooling para reducción espacial.  
   - Se aplicó regularización L2 en las capas convolucionales para evitar el sobreajuste.  
   - Se añadió una capa densa con activación ReLU seguida de dropout (0.5) para mejorar la robustez del modelo.  
   - La capa final es una capa densa con activación softmax para clasificación en 10 categorías.  

3. **Compilación del modelo:**  
   - El modelo se compiló usando el optimizador Adam con una tasa de aprendizaje inicial de 0.001.  
   - Se usó la función de pérdida categorical_crossentropy, adecuada para problemas multiclase con etiquetas one-hot.  

4. **Configuración de callbacks:**  
   - Se incluyó EarlyStopping para detener el entrenamiento cuando la pérdida de validación dejara de mejorar tras 5 épocas consecutivas, evitando el sobreentrenamiento.  
   - ReduceLROnPlateau ajustó la tasa de aprendizaje disminuyéndola automáticamente al estancarse la métrica de validación, mejorando la convergencia.  
   - Un callback personalizado registró la evolución del learning rate para su visualización posterior.  

5. **Entrenamiento:**  
   - Se entrenó el modelo durante un máximo de 50 épocas con un batch size de 64, usando un 20% de los datos para validación.  
   - Los callbacks monitorearon el entrenamiento para optimizar el proceso y evitar el sobreajuste.  

6. **Evaluación y visualización:**  
   - Se evaluó la precisión y pérdida en el conjunto de prueba para medir la generalización del modelo.  
   - Se graficaron las curvas de precisión, pérdida y tasa de aprendizaje para analizar el comportamiento del entrenamiento y ajuste del learning rate.  

---

# 2. Configuración del entorno

---

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LambdaCallback
import matplotlib.pyplot as plt

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`cargar_y_preprocesar_datos()`** 
Carga el dataset CIFAR-10, normaliza las imágenes y convierte las etiquetas a formato one-hot.

---

##### Decisiones de diseño:

#### Elección del dataset CIFAR-10

- El dataset CIFAR-10 fue elegido por ser un conjunto de imágenes en color ampliamente utilizado para tareas de clasificación en visión por computadora. Su tamaño moderado y diversidad de 10 clases lo hacen adecuado para entrenar y evaluar modelos de redes neuronales convolucionales, permitiendo experimentar con técnicas de regularización y optimización sin requerir recursos computacionales excesivos. Esto lo convierte en una opción ideal para proyectos educativos y pruebas rápidas de arquitecturas, asegurando un buen equilibrio entre complejidad y practicidad.

---

In [ ]:
def cargar_y_preprocesar_datos():
    """
    Carga el dataset CIFAR-10 y realiza el preprocesamiento necesario para el entrenamiento.

    Específicamente:
    - Descarga y separa los datos en conjuntos de entrenamiento y prueba.
    - Normaliza los valores de píxeles a un rango [0, 1] para mejorar la convergencia.
    - Convierte las etiquetas a formato one-hot encoding para clasificación multiclase.

    Returns:
        tuple: Cuatro arrays numpy:
            - x_train: Imágenes de entrenamiento normalizadas (float32).
            - y_train: Etiquetas de entrenamiento codificadas one-hot.
            - x_test: Imágenes de prueba normalizadas (float32).
            - y_test: Etiquetas de prueba codificadas one-hot.
    """
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

**Bloque 2:** Creación, compilación y entrenamiento del modelo.

- **`construir_modelo()`** 
Define y arma la arquitectura CNN con capas convolucionales, regularización L2 y dropout.

- **`compilar_modelo()`** 
Compila el modelo con el optimizador Adam y la función de pérdida para clasificación multiclase.

- **`obtener_callbacks()`** 
Crea los callbacks para early stopping, reducción de learning rate y registro de la tasa de aprendizaje.

---

##### Decisiones de diseño:

#### Arquitectura del modelo detallada

- La arquitectura propuesta combina dos capas convolucionales con activación ReLU y regularización L2 para extraer características relevantes mientras se controla el sobreajuste mediante la penalización de pesos grandes. Las capas MaxPooling reducen la dimensionalidad espacial de las representaciones, facilitando el aprendizaje y disminuyendo la complejidad computacional. La capa Flatten transforma las características extraídas en un vector para ser procesado por las capas densas. La capa densa intermedia con activación ReLU permite al modelo aprender combinaciones no lineales de las características, mientras que la capa Dropout introduce aleatoriedad en la desconexión de neuronas durante el entrenamiento para mejorar la robustez y evitar el sobreajuste. Finalmente, la capa de salida con activación softmax genera una distribución de probabilidades para la clasificación multiclase, ajustándose al problema específico del dataset.

---

In [ ]:
def construir_modelo(tamaño_entrada=(32, 32, 3), num_clases=10):
    """
    Construye un modelo de red neuronal convolucional (CNN) para clasificación de imágenes.

    Arquitectura:
    - Dos capas convolucionales con activación ReLU y regularización L2.
    - Capas MaxPooling para reducción espacial.
    - Capa Flatten para vectorizar las características.
    - Capa densa intermedia con activación ReLU.
    - Capa Dropout para evitar sobreajuste.
    - Capa de salida densa con activación softmax para clasificación multiclase.

    Args:
        tamaño_entrada (tuple): Dimensiones de las imágenes de entrada (alto, ancho, canales).
        num_clases (int): Cantidad de clases para la clasificación.

    Returns:
        keras.Model: Modelo CNN no compilado listo para entrenamiento.
    """
    modelo = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same',
               kernel_regularizer=keras.regularizers.l2(0.001), input_shape=tamaño_entrada),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same',
               kernel_regularizer=keras.regularizers.l2(0.001)),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_clases, activation='softmax')
    ])

    return modelo


def compilar_modelo(modelo, tasa_aprendizaje=0.001):
    """
    Compila el modelo con el optimizador Adam y función de pérdida para clasificación multiclase.

    Parámetros configurables:
    - Tasa de aprendizaje inicial para el optimizador Adam.

    Args:
        modelo (keras.Model): Modelo no compilado.
        tasa_aprendizaje (float): Learning rate para el optimizador Adam.

    Returns:
        keras.Model: Modelo compilado listo para entrenamiento.
    """
    optimizador = keras.optimizers.Adam(learning_rate=tasa_aprendizaje)
    modelo.compile(
        optimizer=optimizador,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return modelo


def obtener_callbacks(modelo):
    """
    Crea una lista de callbacks para mejorar el entrenamiento y seguimiento.

    Callbacks incluidos:
    - EarlyStopping: Detiene el entrenamiento si la pérdida de validación no mejora tras cierta paciencia.
    - ReduceLROnPlateau: Reduce la tasa de aprendizaje si la métrica monitorizada se estanca.
    - LambdaCallback: Registra el learning rate al final de cada época para visualizar su evolución.

    Args:
        modelo (keras.Model): Modelo compilado cuyo optimizador se monitoriza para el learning rate.

    Returns:
        tuple:
            - lista_callbacks (list): Lista con los callbacks definidos.
            - lista_lr (list): Lista vacía que se llenará con los valores del learning rate durante el entrenamiento.
    """
    lista_lr = []

    seguimiento_lr = LambdaCallback(on_epoch_end=lambda epoca, logs: lista_lr.append(
        float(keras.backend.get_value(modelo.optimizer.lr))
    ))

    parada_temprana = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    reduccion_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )

    return [parada_temprana, reduccion_lr, seguimiento_lr], lista_lr

**Bloque 3:** Visualización de resultados.

- **`graficar_entrenamiento()`** 
Genera gráficos de precisión, pérdida y evolución del learning rate durante el entrenamiento.

---

##### Decisiones de diseño:

#### Elección de parámetros para graficar

- Se eligieron para graficar la precisión y pérdida tanto en entrenamiento como en validación con el fin de monitorear el desempeño del modelo y detectar posibles señales de sobreajuste o subajuste durante el proceso de entrenamiento. La precisión muestra qué tan bien el modelo clasifica correctamente las muestras, mientras que la pérdida refleja qué tan lejos están las predicciones de los valores reales, ofreciendo una medida más sensible a la calidad del ajuste. Además, se incluyó la evolución del learning rate por época para visualizar cómo los callbacks de ajuste automático de tasa de aprendizaje impactan en la convergencia del modelo, facilitando la interpretación de cambios en el rendimiento asociados a modificaciones en este hiperparámetro crítico.

---

In [ ]:
def graficar_entrenamiento(historial, lista_lr):
    """
    Grafica las curvas de precisión, pérdida y tasa de aprendizaje durante el entrenamiento.

    Parámetros graficados:
    - Precisión en entrenamiento y validación.
    - Pérdida en entrenamiento y validación.
    - Evolución del learning rate por época.

    Args:
        historial (History): Objeto retornado por model.fit() con historial del entrenamiento.
        lista_lr (list): Valores del learning rate registrados por época.
    """
    plt.figure(figsize=(14, 4))

    plt.subplot(1, 3, 1)
    plt.plot(historial.history['accuracy'], label='Entrenamiento')
    plt.plot(historial.history['val_accuracy'], label='Validación')
    plt.title("Precisión")
    plt.xlabel("Épocas")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(historial.history['loss'], label='Entrenamiento')
    plt.plot(historial.history['val_loss'], label='Validación')
    plt.title("Pérdida")
    plt.xlabel("Épocas")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 3)
    plt.plot(lista_lr, marker='o')
    plt.title("Learning Rate")
    plt.xlabel("Épocas")
    plt.ylabel("LR")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución.

- **`main()`** 
Ejecuta el flujo completo: carga datos, crea y entrena el modelo, evalúa y visualiza resultados.

---

In [ ]:
def main():
    """
    Función principal que ejecuta el flujo completo de:
    - Carga y preprocesamiento de datos.
    - Construcción y compilación del modelo.
    - Configuración de callbacks.
    - Entrenamiento del modelo.
    - Evaluación sobre el conjunto de prueba.
    - Visualización de métricas y tasa de aprendizaje.
    """
    x_train, y_train, x_test, y_test = cargar_y_preprocesar_datos()

    modelo = construir_modelo()
    modelo = compilar_modelo(modelo)

    callbacks, lista_lr = obtener_callbacks(modelo)

    historial = modelo.fit(
        x_train, y_train,
        validation_split=0.2,
        epochs=50,
        batch_size=64,
        callbacks=callbacks,
        verbose=1
    )

    perdida_test, accuracy_test = modelo.evaluate(x_test, y_test, verbose=0)
    print(f"Accuracy en test: {accuracy_test:.4f}")

    graficar_entrenamiento(historial, lista_lr)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

>Nota: Los resultados estan basados en valores especificos obtenidos en una ejecución aleatoria, por lo que dichos valores de accuracy por ejemplo, podrián variar en otras ejecuciones. De igual forma, los resultados estan hechos con el fin de representar el significado del uso de los métodos de regularización y optimización en redes neuronales.

---

## Análisis de resultados

- El entrenamiento comienza con una accuracy del 39 % y una val_accuracy del 52 %, lo que indica que el modelo empieza a aprender desde el inicio, considerando que en CIFAR-10 el azar puro daría alrededor de un 10 %. A lo largo de las primeras 23 épocas, la accuracy de entrenamiento aumenta del 39 % al 80 %, mientras que la de validación pasa del 52 % al 73 %. Sin embargo, entre las épocas 13 y 16 la val_accuracy se mantiene alrededor del 71 % mientras la accuracy de entrenamiento sigue mejorando, lo que señala el inicio de un sobreajuste. En la época 17, el callback ReduceLROnPlateau reduce el learning rate a 0.0005, lo que permite desbloquear la mejora y alcanzar un 73.2 % de val_accuracy. Al finalizar, el modelo logra un 80.5 % de accuracy en entrenamiento, 73.2 % en validación y 72.36 % en el conjunto de prueba, resultado consistente con la validación y sin indicios de fuga de datos.

- El modelo presenta una generalización aceptable, con una diferencia de aproximadamente 7–8 puntos porcentuales entre la accuracy de entrenamiento y la de validación en la última época, lo cual es normal para una CNN pequeña en CIFAR-10. El uso del Learning Rate Scheduler resultó efectivo, ya que las reducciones de la tasa de aprendizaje en las épocas 17 y 22 evitaron el estancamiento en la mejora de la validación. Además, la regularización mediante L2 y Dropout contribuyó a contener el sobreajuste, aunque la arquitectura, por su simplicidad, tiene ciertas limitaciones para capturar patrones más complejos del dataset.

- Considerando que se trata de una CNN pequeña con solo dos capas convolucionales, un accuracy de prueba del 72 % es consistente y competitivo para CIFAR-10 sin técnicas avanzadas de aumento de datos

---

### ¿Qué técnica tuvo mayor impacto?

Las técnicas que tuvieron mayor impacto en la mejora del rendimiento y la generalización del modelo fueron:

- **Regularización L2** en las capas convolucionales, que ayuda a reducir el sobreajuste penalizando pesos grandes y estabilizando el entrenamiento.  
- **Dropout** en la capa densa final, que promueve una representación más robusta al impedir la dependencia excesiva en neuronas específicas.  
- **Callbacks de EarlyStopping y ReduceLROnPlateau**, que detienen el entrenamiento cuando la pérdida de validación deja de mejorar y ajustan automáticamente la tasa de aprendizaje, respectivamente.  
Estas técnicas combinadas permitieron que el modelo aprendiera adecuadamente sin caer en sobreajuste ni quedar atrapado en mínimos locales.

### ¿Cómo se evitó el sobreajuste?

El sobreajuste se evitó mediante varias estrategias implementadas en el modelo:

- **Regularización L2 (weight decay)** para penalizar pesos grandes y favorecer modelos más simples.  
- **Dropout** para desconectar aleatoriamente neuronas durante el entrenamiento, evitando la dependencia excesiva de conexiones específicas.  
- **EarlyStopping**, que interrumpe el entrenamiento cuando la métrica de validación deja de mejorar, evitando un entrenamiento excesivo.  
- **Separación de un conjunto de validación** para monitorear el rendimiento del modelo en datos no vistos durante el entrenamiento.

Estas medidas evitaron que el modelo memorizara el conjunto de entrenamiento y favorecieron una mejor generalización a datos nuevos.

### ¿Qué se haría diferente si el dataset fuese más grande?

Si el dataset fuera considerablemente más grande, se considerarían las siguientes modificaciones para aprovechar mejor los datos y recursos:

- Implementar **Data Augmentation** para aumentar artificialmente la diversidad de las imágenes y mejorar la capacidad de generalización del modelo.  
- Ajustar la intensidad de la **regularización L2** y la tasa de **dropout**, ya que con más datos el riesgo de sobreajuste disminuye, permitiendo modelos más complejos.  
- Incrementar la profundidad y complejidad del modelo, añadiendo más capas convolucionales y filtros para capturar patrones más complejos.  
- Utilizar optimizadores o programadores de tasa de aprendizaje más sofisticados, como **Lookahead** o **Cosine Annealing**, para mejorar la convergencia en entrenamientos prolongados.  
- Aplicar técnicas de **entrenamiento distribuido** o usar hardware especializado como GPUs o TPUs para acelerar el proceso.  
- Incorporar **Batch Normalization** para estabilizar y acelerar el entrenamiento en arquitecturas más profundas.  
- Implementar **checkpointing** para guardar periódicamente el estado del modelo y evitar pérdidas de progreso en entrenamientos largos.

---
---

## Pruebas adicionales con Batch Normalization y Data Augmentation

- Se realizaron experimentos incorporando técnicas adicionales como **Batch Normalization** y **Data Augmentation**, así como aumentando la complejidad del modelo con el fin de potenciar su rendimiento. Estas mejoras demostraron ser efectivas para aumentar la precisión y reducir la pérdida, pero también provocaron un incremento significativo en el tiempo de entrenamiento, llegando a superar los 22 minutos en hardware convencional.

- Dado que el contexto de la tarea cuenta con limitaciones de tiempo y recursos computacionales, se decidió optar por un modelo más sencillo que mantuviera un buen balance entre desempeño y eficiencia. No obstante, estas técnicas avanzadas quedan como posibles mejoras para futuras implementaciones o para entornos con mayor capacidad de cómputo.

- Es importante destacar los resultados alcanzados durante esta prueba mejorada, que incluyen:
    - Época final: 47/50  
    - Pérdida (loss) en entrenamiento: 0.5162  
    - Precisión (accuracy) en entrenamiento: 84.81%  
    - Pérdida en validación: 0.5707  
    - Precisión en validación: 83.78%  
    - Learning rate ajustado: 3.1250e-05  
    - Precisión final en test: 84.17%  

- Estos experimentos con Batch Normalization y Data Augmentation son precisamente dos de las estrategias que se plantearon en la sección ¿Qué haría diferente si el dataset fuese más grande?. Aunque el dataset original no aumentó en tamaño, el Data Augmentation permitió simular un conjunto más diverso, y la Batch Normalization se probó como técnica de estabilización en arquitecturas más profundas. Los resultados —un aumento de la precisión en validación del 73.2 % al 83.78 %— demuestran que estas ideas no son solo teóricas, sino que ya han mostrado un impacto positivo. Con un dataset realmente mayor y más capacidad de cómputo, es probable que estas técnicas ofrezcan aún mejores mejoras de generalización.

---